### Semana 6, Día 2

¡Estamos a punto de crear y usar nuestro propio servidor y cliente MCP!

Es bastante sencillo, pero no es supersencillo. Lo emocionante de MCP radica en lo fácil que es compartir y usar otros servidores MCP; crear uno propio requiere algo de trabajo.

Revisemos código Python creado principalmente por un equipo de ingeniería muy dedicado:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [5]:
account = Account.get("Juan Gabriel")
account

Account(name='juan gabriel', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [6]:
account.buy_shares("AMZN", 3, "Porque este sitio web de libros parece prometedor")

'Completado. Últimos detalles:\n{"name": "juan gabriel", "balance": 9774.55, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 75.15, "timestamp": "2025-07-16 16:47:58", "rationale": "Porque este sitio web de libros parece prometedor"}], "portfolio_value_time_series": [["2025-07-16 16:47:58", 9849.55]], "total_portfolio_value": 9849.55, "total_profit_loss": -150.45000000000073}'

In [7]:
account.report()

'{"name": "juan gabriel", "balance": 9774.55, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 75.15, "timestamp": "2025-07-16 16:47:58", "rationale": "Porque este sitio web de libros parece prometedor"}], "portfolio_value_time_series": [["2025-07-16 16:47:58", 9849.55], ["2025-07-16 16:48:31", 9951.55]], "total_portfolio_value": 9951.55, "total_profit_loss": -48.45000000000073}'

In [8]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 75.15,
  'timestamp': '2025-07-16 16:47:58',
  'rationale': 'Porque este sitio web de libros parece prometedor'}]

### Ahora escribimos un servidor MCP y lo usamos directamente

In [9]:
# Ahora usemos nuestro servidor de cuentas como servidor MCP

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [10]:
mcp_tools

[Tool(name='get_balance', description='Obtiene el saldo en efectivo de la cuenta indicada.\n\n    Args:\n        name: El nombre del titular de la cuenta\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Obtiene las tenencias de la cuenta indicada.\n\n    Args:\n        name: El nombre del titular de la cuenta\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description='Compra acciones de una empresa.\n\n    Args:\n        name: El nombre del titular de la cuenta\n        symbol: El símbolo de la acción\n        quantity: La cantidad de acciones a comprar\n        rationale: La razón de la compra y su relación con la estrategia de la cuenta\n    ', inputSchema={'properties'

In [11]:
instructions = "Puedes gestionar una cuenta para un cliente y responder preguntas sobre la cuenta."
request = "Me llamo Juan Gabriel y mi cuenta está a nombre de Juan Gabriel. ¿Cuál es mi saldo y mis tenencias?"
model = "gpt-4.1-mini"

In [12]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))
    


Juan Gabriel, tu saldo actual en efectivo es de 9,774.55 unidades monetarias. En cuanto a tus tenencias, posees 3 acciones de Amazon (AMZN). ¿Quieres hacer alguna operación o necesitas información adicional?

### Vamos a construir nuestro propio cliente MCP

In [13]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Obtiene el saldo en efectivo de la cuenta indicada.\n\n    Args:\n        name: El nombre del titular de la cuenta\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Obtiene las tenencias de la cuenta indicada.\n\n    Args:\n        name: El nombre del titular de la cuenta\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description='Compra acciones de una empresa.\n\n    Args:\n        name: El nombre del titular de la cuenta\n        symbol: El símbolo de la acción\n        quantity: La cantidad de acciones a comprar\n        rationale: La razón de la compra y su relación con la estrategia de la cuenta\n    ', inputSchema={'properties': 

In [14]:
request = "Me llamo Juan Gabriel y mi cuenta está a nombre de Juan Gabriel. ¿Cuál es mi saldo?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

El saldo en efectivo de la cuenta a nombre de Juan Gabriel es de 9,774.55 unidades monetarias. ¿Deseas que consulte algo más sobre tu cuenta?

In [ ]:
context = await read_accounts_resource("Juan Gabriel")
print(context)

{"name": "juan%20gabriel", "balance": 10000.0, "strategy": "", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-16 17:18:57", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}


In [16]:
from accounts import Account
Account.get("Juan Gabriel").report()

'{"name": "juan gabriel", "balance": 9774.55, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 75.15, "timestamp": "2025-07-16 16:47:58", "rationale": "Porque este sitio web de libros parece prometedor"}], "portfolio_value_time_series": [["2025-07-16 16:47:58", 9849.55], ["2025-07-16 16:48:31", 9951.55], ["2025-07-16 17:19:31", 10035.55]], "total_portfolio_value": 10035.55, "total_profit_loss": 35.54999999999927}'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Ejercicios</h2>
            <span style="color:#ff7800;">¡Crea tu propio servidor MCP! Crea una función simple que devuelva la fecha actual y expónla como herramienta para que un agente pueda indicarte la fecha de hoy.<br/>Ejercicio opcional más complejo: crea un cliente MCP y usa una llamada nativa de OpenAI (sin el SDK de agentes) para usar tu herramienta a través de tu cliente.
            </span>
        </td>
    </tr>
</table>